In [4]:
import pickle
import numpy as np 
from matplotlib import pyplot as plt 

file_head = '../Baselines/EnemyPositionPrediction/checkpoints'
CNN_GRU_5_1_result = '/CNN_GRU_5_1/all/test/test_result'
CNN_GRU_5_2_result = '/CNN_GRU_5_2/all/test/test_result'
CNN_GRU_7_1_result = '/CNN_GRU_7_1/all/test/test_result'
# CNN_GRU_7_2_result = '/CNN_GRU_7_2/all/test/test_result'
CNN_GRU_9_1_result = '/CNN_GRU_9_1/all/test/test_result'
with open(file_head + CNN_GRU_5_1_result,'rb') as f:
    CNN_GRU_5_1_file = pickle.load(f)
with open(file_head + CNN_GRU_5_2_result,'rb') as f:
    CNN_GRU_5_2_file = pickle.load(f)
# with open(file_head + CNN_GRU_7_1_result,'rb') as f:
#     Res_tensor_vector_result_file = pickle.load(f)
with open(file_head + CNN_GRU_7_2_result,'rb') as f:
    CNN_GRU_7_2_file = pickle.load(f)
with open(file_head + CNN_GRU_9_1_result,'rb') as f:
    CNN_GRU_9_1_file = pickle.load(f)

In [3]:
def get_max_acc(result_file, offset=0):
    acc_n = []
    for dic in result_file:
        acc = [[] for _ in range(7)]
        for piece in range(6):
            action_pre=dic['action_pre_per_replay'][piece]
            action_gt = dic['action_gt_per_replay'][piece]
            for action_pre_per_replay, action_gt_per_replay in zip(action_pre, action_gt):
                acc_per_replay = np.mean(action_pre_per_replay[offset:] == action_gt_per_replay[offset:])
                acc[piece].append(acc_per_replay)

        for piece in range(6):
            acc[piece] = np.array(acc[piece]).mean()
        acc[6] = np.array(acc[:6]).mean()
        acc_n.append(acc)
    acc_n = np.array(acc_n)    
    max_row = np.argmax(acc_n[:,-1])
    ##################################
    steps = 10
    dic_max = result_file[max_row]
    
    acc_n_step = []
    action_pre_result, action_gt_result = [[] for _ in range(steps)], [[] for _ in range(steps)]
    for piece in range(6):
        action_pre = dic_max['action_pre_per_replay'][piece]
        action_gt = dic_max['action_gt_per_replay'][piece]
        for action_pre_per_replay, action_gt_per_replay in zip(action_pre, action_gt):
            if len(action_pre_per_replay[offset:]) < steps:
                continue

            step = len(action_pre_per_replay[offset:]) // steps
            for s in range(steps):
                action_pre_result[s].append(action_pre_per_replay[offset:][s * step:(s + 1) * step])
                action_gt_result[s].append(action_gt_per_replay[offset:][s * step:(s + 1) * step])

        X = np.arange(steps)
        Y = np.zeros(steps)
        for idx, (action_pres, action_gts) in enumerate(zip(action_pre_result, action_gt_result)):

            action_pres_np = np.hstack(action_pres)
            action_gts_np = np.hstack(action_gts)

            Y[idx] = np.mean(action_pres_np == action_gts_np)
        acc_n_step.append(Y)
    return acc_n[max_row, :], np.array(acc_n_step)

In [5]:
CNN_GRU_5_1_acc_max, CNN_GRU_5_1_acc_n_step = get_max_acc(CNN_GRU_5_1_file, offset=0)
CNN_GRU_5_2_acc_max, CNN_GRU_5_2_acc_n_step = get_max_acc(CNN_GRU_5_2_file, offset=0)
# Res_tensor_vector_acc_max, Res_tensor_vector_acc_n_step = get_max_acc(Res_tensor_vector_result_file, offset=9)
CNN_GRU_7_2_acc_max, CNN_GRU_7_2_acc_n_step = get_max_acc(CNN_GRU_7_2_file, offset=0)
CNN_GRU_9_1_acc_max, CNN_GRU_9_1_acc_n_step = get_max_acc(CNN_GRU_9_1_file, offset=0)

# print(Res_tensor_acc_n_step)
print(CNN_GRU_5_1_acc_max, CNN_GRU_5_2_acc_max, CNN_GRU_7_2_acc_max, CNN_GRU_9_1_acc_max)

[0.2799726  0.26349458 0.29117176 0.27810336 0.36723264 0.35539351
 0.30589474] [0.26850221 0.24133785 0.28593288 0.27097649 0.37553874 0.35156581
 0.29897566] [0.29999548 0.25379487 0.289165   0.2711406  0.36857819 0.34493772
 0.30460198] [0.30341162 0.27243758 0.28257847 0.25808266 0.37757332 0.30395573
 0.29967323]


In [9]:
import numpy as np
a = np.arange(16).reshape(2,2,4)
b = np.array([1,2,3,4])
print(b*a)

[[[ 0  2  6 12]
  [ 4 10 18 28]]

 [[ 8 18 30 44]
  [12 26 42 60]]]
